# Tabular example

In [ ]:
from fastai.tabular import *  # Quick accesss to tabular functionality

Tabular data should be in a Pandas `DataFrame`.

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [ ]:
df['salary'].unique()

array(['>=50k', '<50k'], dtype=object)

In [ ]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [ ]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [ ]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [ ]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [ ]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target
Private,Some-college,Never-married,Handlers-cleaners,Own-child,White,False,-1.5090,0.2360,-0.0312,<50k
Private,Some-college,Divorced,Exec-managerial,Not-in-family,White,False,-0.4828,0.0802,-0.0312,<50k
Local-gov,HS-grad,Separated,Other-service,Not-in-family,Black,False,0.2502,0.3442,-0.4224,<50k
Private,Assoc-voc,Married-civ-spouse,Craft-repair,Husband,White,False,0.9098,-0.8595,0.3599,<50k
Private,HS-grad,Never-married,Sales,Not-in-family,White,False,-1.0692,-0.6139,-0.4224,<50k
Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,False,-0.3362,0.2229,1.1422,<50k
Private,Bachelors,Never-married,Farming-fishing,Own-child,White,False,-0.9959,-0.8271,1.1422,>=50k
State-gov,Bachelors,Divorced,Adm-clerical,Unmarried,White,False,0.6899,-1.3176,1.1422,<50k
Private,HS-grad,Married-civ-spouse,Adm-clerical,Wife,Black,False,-1.0692,-1.3076,-0.4224,<50k
Private,Masters,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,False,-0.3362,-0.8518,1.5334,>=50k


In [ ]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.358096,0.370009,0.830000


## Inference

In [ ]:
row = df.iloc[0]

In [ ]:
learn.predict(row)

(Category <50k, tensor(0), tensor([0.5142, 0.4858]))